Segun codigo de clases

In [10]:
import yfinance as yf
import numpy as np
from scipy.stats import norm
import datetime

# 1. Obtener datos históricos y cálculos de precio actual, volatilidad y tasa libre de riesgo

# Símbolo del Banco Santander Chile en Yahoo Finance
symbol = 'BSAC'

# Fechas de inicio y fin
start_date = datetime.datetime(2023, 1, 1)
end_date = datetime.datetime(2024, 9, 25)  # Fecha actual fija

# Obtener datos históricos
stock_data = yf.download(symbol, start=start_date, end=end_date)

# Obtener el precio actual
current_price = stock_data['Adj Close'][-1]
print(f"El precio actual de una acción del Banco Santander Chile es: {current_price}")

# Obtener la fecha de vencimiento
fecha_actual = datetime.datetime(2024, 9, 25)  # Fecha fija de hoy
fecha_vencimiento = datetime.datetime(2024, 12, 31)  # Fecha de vencimiento fija

diferencia = fecha_vencimiento - fecha_actual
dias_hasta_vencimiento = diferencia.days
años_hasta_vencimiento = dias_hasta_vencimiento / 252  # Asumiendo 252 días hábiles

# Obtener la volatilidad histórica
historical_data = yf.download(symbol, start=start_date, end=end_date)
historical_data['Daily Returns'] = historical_data['Adj Close'].pct_change()  # Calcular rendimientos diarios
volatility = np.std(historical_data['Daily Returns']) * np.sqrt(252)  # Volatilidad anualizada
print(f'Volatilidad histórica del Banco Santander Chile: {volatility:.4f}')

# Tasa de interés libre de riesgo
symbol_bono = "^TNX"  # Usaremos bonos del Tesoro de EE.UU. como aproximación
historical_data_bono = yf.download(symbol_bono, start=start_date, end=end_date)
historical_data_bono['Daily Returns'] = historical_data_bono['Adj Close'].pct_change()
tasa_libre_riesgo = np.mean(historical_data_bono['Daily Returns']) * 252  # Tasa anualizada
print(f'Tasa de interés libre de riesgo compuesta continuamente: {tasa_libre_riesgo:.4f}')

# Precio spot actual (S0)
data = yf.Ticker(symbol)
S0 = data.history(period="1d")["Close"].iloc[-1]
print(f"El S0 será: {S0}")

# 2. Cálculo del precio de la opción usando el modelo de Black-Scholes

def black_scholes_option_price(S, K, T, r, sigma, option_type='call'):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if option_type == 'call':
        option_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        option_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("option_type debe ser 'call' o 'put'.")
    return option_price

# Parámetros para Black-Scholes
K = current_price  # Precio de ejercicio
T = años_hasta_vencimiento  # Tiempo hasta la expiración
r = tasa_libre_riesgo  # Tasa libre de riesgo
sigma = volatility  # Volatilidad

# Cálculo del precio de opción de compra y venta
call_option_price = black_scholes_option_price(S0, K, T, r, sigma, option_type='call')
put_option_price = black_scholes_option_price(S0, K, T, r, sigma, option_type='put')
print(f'Precio de la opción de compra (call) según Black-Scholes: {call_option_price:.4f}')
print(f'Precio de la opción de venta (put) según Black-Scholes: {put_option_price:.4f}')

# 3. Cálculo del precio de la opción usando el modelo binomial

def binomial_option_pricing(S, K, T, r, sigma, n, option_type='call'):
    dt = T / n
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)
    option_prices = np.zeros((n + 1,))

    # Calcular precios en el último paso
    for i in range(n + 1):
        if option_type == 'call':
            option_prices[i] = max(0, S * (u ** i) * (d ** (n - i)) - K)
        elif option_type == 'put':
            option_prices[i] = max(0, K - S * (u ** i) * (d ** (n - i)))

    # Calcular los precios hacia atrás
    for j in range(n, 0, -1):
        for i in range(j):
            option_prices[i] = np.exp(-r * dt) * (p * option_prices[i + 1] + (1 - p) * option_prices[i])

    return option_prices[0]

# Parámetros para el modelo binomial
n = 100  # Número de pasos
# Cálculo del precio de opción de compra y venta con modelo binomial
call_option_price_binomial = binomial_option_pricing(S0, K, T, r, sigma, n, option_type='call')
put_option_price_binomial = binomial_option_pricing(S0, K, T, r, sigma, n, option_type='put')

print(f'Precio de la opción de compra (call) según el modelo binomial: {call_option_price_binomial:.4f}')
print(f'Precio de la opción de venta (put) según el modelo binomial: {put_option_price_binomial:.4f}')


[*********************100%***********************]  1 of 1 completed
<ipython-input-10-6c594d64822d>:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_price = stock_data['Adj Close'][-1]
[*********************100%***********************]  1 of 1 completed


El precio actual de una acción del Banco Santander Chile es: 20.8799991607666
Volatilidad histórica del Banco Santander Chile: 0.2552


[*********************100%***********************]  1 of 1 completed


Tasa de interés libre de riesgo compuesta continuamente: 0.0284
El S0 será: 20.760000228881836
Precio de la opción de compra (call) según Black-Scholes: 1.3610
Precio de la opción de venta (put) según Black-Scholes: 1.2540
Precio de la opción de compra (call) según el modelo binomial: 1.3616
Precio de la opción de venta (put) según el modelo binomial: 1.2546


In [1]:
from sympy import *
init_printing()

La estructura financiera es la siguiente:

Dos periodos $t=0,1$ y 2 estados de la naturaleza en $t=1$: bueno (u) y malo (d)

Dos activos:
- libre de riesgo (bono) con valor inicial $B$ y retorno $\tilde r>1$
- riesgoso (acción) con valor inicial $S_0$ y retornos $u>1$ ó $1>d$

Derivado: opción de compra Europea de la acción (se ejerce solo al final)
- $K$ precio de ejercicio
- $S_1$ valor del activo al final
- $\max\{0,S_1-K\}$ retorno para $S_1\in\{uS_0,dS_0\}$

1. ¿Cuál es el precio de los activos no derivados?
- De esta forma el precio del bono es $B$ y el precio de la acción es $S_0$.
2. ¿Cuál es el retorno de los activos no derivados?
- El retorno del bono es $\tilde rB$ tanto en el estado bueno como en el estado malo
- El retorno de la acción es $uS_0$ en el estado bueno y $dS_0$ en el estado malo
3. ¿Cuál es el retorno de la opción?
- RESPONDER

Inicialmente nos concentraremos en los activos no derivados. Definimos $W=\left[-q'\atop R\right]$ donde los precios de los activos son $q=(q_1,q_2)=(B,S_0)$ y los retornos son $R=(R_1,R_2)$.
1. Encuentre la expresión de $R_1$ y $R_2$.
- RESPONDER
2. Encuentre $W$.
- RESPONDER


In [2]:
# Parámetros Bono
r, B = symbols('r B')
# Parámetros Acción
u, d, S0 = symbols('u d S0')

r, B = 0.01, 100
# Parámetros de la acción (modificado)
u, d, S0 = 1.006, 1.003, 20.15  # u y d calculados con precio open y close y S0


accion = S0

retorno_accionU = u * S0
retorno_accionD = d * S0
(accion,retorno_accionU,retorno_accionD)

(20.15, 20.270899999999997, 20.210449999999998)

In [3]:
W = Matrix([[-B, -S0], [r*B, u*S0],[r*B, d*S0]])
W

⎡-100   -20.15 ⎤
⎢              ⎥
⎢1.0   20.2709 ⎥
⎢              ⎥
⎣1.0   20.21045⎦

El Teorema Fundamental de Valoración de Activos ([Harrison and Kreps (1979)](https://www.sciencedirect.com/science/article/pii/0022053179900437)) establece que la estructura financiera  es libre de arbitraje si y sólo si existe un $\pi\in\mathbb{R}^S_{++}$ tal que

(Condición) $[1,\pi']W=0$.

Para utilizar este resultado, definamos el vector $[1,\pi']$.

In [4]:
pi_u, pi_d = symbols('pi_u pi_d')
pi = Matrix([[1],[pi_u],[pi_d]])
pi

⎡ 1 ⎤
⎢   ⎥
⎢πᵤ ⎥
⎢   ⎥
⎣π_d⎦

Reemplazando en la (Condición) obtenemos el siguiente sistema de ecuaciones:

In [5]:
NA=pi.transpose()*W
NA

[1.0⋅π_d + 1.0⋅πᵤ - 100  20.21045⋅π_d + 20.2709⋅πᵤ - 20.15]

Podemos utilizar un _solver_ algebraico para encontrar expresiones generales de $\pi_1$ y $\pi_2$ en función de los parámetros $u,d$ y $r$:

In [6]:
# Solver tutorial
# https://docs.sympy.org/latest/modules/solvers/solvers.html
piNA=solve([NA[0],NA[1]],[pi_u,pi_d],dict=True)
piNA

[{π_d: 33200.0, πᵤ: -33100.0}]

## Inclusión del derivado

Ahora extendamos la matriz $W$ para incorporar la opción.

1. Encuentre la expresión para $W$ agregando los retornos y el precio de la opción denotado por $q_3$.

In [7]:
# Parámetros Opción (modificado)
q3 = symbols('q3')
K = 21  # Precio de ejercicio de la opción, ajustado a 21 basado en el máximo rango del precio (52 Week Range)
# Parámetros Bono
r, B = 1.01, 100  # Ajuste de los parámetros del bono
# Parámetros Acción
u, d, S0 = 1.003, 1.005, 20.15  # Ajuste de los parámetros de la acción
W = Matrix([[-B, -S0, -q3], [1.01*B, u*S0, max(0, u*S0-K)], [1.01*B, d*S0, max(0, d*S0-K)]])
W


⎡-100    -20.15   -q₃⎤
⎢                    ⎥
⎢101.0  20.21045   0 ⎥
⎢                    ⎥
⎣101.0  20.25075   0 ⎦

Utilice la (Condición) del Teorema Fundamental reemplazando los valores encontrados anteriormente para $\pi$ para definir una expresión para $q_3$.

In [8]:
Prices = Matrix([[1], [piNA[0][pi_u]], [piNA[0][pi_d]]]).transpose() * W
Prices.subs([(d, 0.5), (u, 1.5), (r, 0.01)])
Prices

[10000.0  3338.85499999998  -q₃]

In [9]:
Prices.subs([(d,0.5),(u,1.5),(r,0.01)])

[10000.0  3338.85499999998  -q₃]

## Aplicación (Grupos)

Encuentre valores de $r,B,u,d,S_0$ para valorar una opción.

# Anexo
Para más manipulaciones simbólicas ver [este cuaderno.](https://colab.research.google.com/github/jrjohansson/scientific-python-lectures/blob/master/Lecture-5-Sympy.ipynb#scrollTo=GzKR97oUjW8T)